# Machine Learning for UFC dataset


## Imports and installs

In [1]:
!pip install rfpimp
!pip install plotly

  Created wheel for rfpimp: filename=rfpimp-1.3.4-cp36-none-any.whl size=10086 sha256=ca47109549fda3efab861617b748f106dd8801bf5efb985955e12f2d2986d9d0
  Stored in directory: /root/.cache/pip/wheels/c7/35/67/4065afd69df4c0be693ecdec8a9c1433a23f80ee60395272ba
Successfully built rfpimp


In [0]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from  sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from  sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn import model_selection
import plotly.express as px
import plotly.graph_objects as go
import pickle

In [3]:
import rfpimp

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.



## Loading and inspecting the data

In [0]:
data = pd.read_csv("data_for_ml.csv")

In [5]:
data.head()

,Unnamed: 0,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,...,R_total_rounds_fought,R_total_time_fought(seconds),R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,weight_class_Women's Bantamweight,weight_class_Women's Featherweight,weight_class_Women's Flyweight,weight_class_Women's Strawweight,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch,Winner_Red,title_bout_True
0,0,5,0.0,4.0,9.200000,6.000000,0.200000,0.000000,62.600000,20.600000,2.600000,2.000000,48.600000,11.200000,0.800000,7.6,5.400000,0.400000,0.000000,65.40,22.600000,0.466000,0.400000,0.80000,0.200000,0.100000,66.400000,23.600000,4.0,1.0,6.400000,4.000000,1.000000,0.60000,51.200000,17.400000,0.600000,0.200000,39.600000,9.400000,...,27.0,742.60,3.0,0.0,2.0,4.0,2.0,0.0,0.0,8.0,162.56,162.56,135.0,31.0,32.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1
1,1,5,0.0,3.0,14.600000,9.100000,11.800000,7.300000,124.700000,42.100000,2.400000,1.900000,112.000000,32.000000,0.000000,12.3,10.200000,0.800000,0.000000,138.90,51.300000,0.399000,0.700000,1.00000,0.500000,0.225000,158.700000,69.600000,3.0,6.0,13.000000,9.300000,12.800000,9.60000,101.700000,32.000000,8.100000,6.900000,97.700000,30.800000,...,25.0,1062.00,2.0,0.0,1.0,2.0,0.0,2.0,0.0,5.0,165.10,167.64,125.0,32.0,31.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,1
2,2,3,0.0,3.0,15.354839,11.322581,6.741935,4.387097,84.741935,38.580645,5.516129,3.806452,67.645161,23.258065,0.645161,14.0,12.193548,0.935484,0.096774,97.00,46.774194,0.496129,0.354839,2.16129,0.677419,0.295484,103.709677,52.548387,8.0,8.0,17.903226,11.870968,8.419355,5.83871,84.548387,38.064516,1.741935,0.935484,67.645161,25.483871,...,33.0,604.40,2.0,0.0,1.0,3.0,3.0,6.0,1.0,14.0,180.34,193.04,155.0,36.0,35.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
3,3,3,0.0,4.0,17.000000,14.000000,13.750000,11.000000,109.500000,48.750000,13.000000,10.500000,116.250000,53.750000,0.500000,3.0,2.500000,0.500000,0.250000,136.25,70.250000,0.550000,0.250000,2.50000,1.250000,0.287500,154.750000,86.750000,4.0,0.0,12.250000,6.000000,6.000000,3.75000,94.250000,26.750000,1.750000,1.250000,82.500000,21.500000,...,20.0,690.25,0.0,0.0,1.0,4.0,1.0,0.0,0.0,6.0,162.56,172.72,135.0,26.0,29.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
4,4,3,0.0,1.0,17.000000,14.500000,2.500000,2.000000,201.000000,59.500000,0.000000,0.000000,184.500000,45.000000,0.000000,2.0,2.000000,0.000000,0.000000,203.50,61.500000,0.310000,0.000000,0.00000,0.000000,0.000000,204.000000,62.000000,1.0,1.0,42.500000,23.500000,0.500000,0.50000,205.000000,89.500000,0.000000,0.000000,152.500000,56.500000,...,7.0,440.75,0.0,0.0,0.0,1.0,2.0,0.0,0.0,3.0,187.96,190.50,264.0,32.0,26.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0


In [6]:
data.shape

(3592, 159)

## Initial steps for ML
Deviding the data

In [0]:
X = data.loc[:, data.columns != 'Winner_Red']

In [0]:
y = data["Winner_Red"]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Machine learning

### Decision Tree Classifier

In [10]:
model=DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
y_pred = model.predict(X_test)

In [12]:
accuracy_score(y_test, y_pred)

0.6349072512647554

In [13]:
precision_score(y_test, y_pred)

0.7289002557544757

In [14]:
recall_score(y_test, y_pred)

0.7206068268015171

In [15]:
f1_score(y_test, y_pred)

0.7247298156389065

In [0]:
model=DecisionTreeClassifier()
scores = cross_validate(model, X, y, cv=3, scoring=('accuracy', 'average_precision', 'recall','f1'))

In [17]:
scores

{'fit_time': array([0.15685678, 0.13578391, 0.17932343]),
 'score_time': array([0.00782847, 0.00711417, 0.00695634]),
 'test_accuracy': array([0.33722871, 0.16541353, 0.66332498]),
 'test_average_precision': array([0.66277129, 0.60536036, 0.66304348]),
 'test_f1': array([0.        , 0.1844898 , 0.79738562]),
 'test_recall': array([0.        , 0.14249685, 1.        ])}

### Random Forest Classifier

In [18]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred = rfc.predict(X_test)

In [20]:
accuracy_score(y_test, y_pred)

0.6804384485666105

In [21]:
precision_score(y_test, y_pred)

0.7003891050583657

In [22]:
recall_score(y_test, y_pred)

0.9102402022756005

In [23]:
f1_score(y_test, y_pred)

0.7916437603078614

In [0]:
rfc_scores = cross_validate(rfc, X, y.values.ravel(), cv=3, scoring=('accuracy', 'average_precision', 'recall','f1'))

In [25]:
rfc_scores

{'fit_time': array([1.15781069, 1.17000055, 1.27894163]),
 'score_time': array([0.05187106, 0.05720305, 0.04639149]),
 'test_accuracy': array([0.38063439, 0.42522974, 0.66081871]),
 'test_average_precision': array([0.72957395, 0.56683919, 0.68871248]),
 'test_f1': array([0.16063348, 0.5394913 , 0.79577465]),
 'test_recall': array([0.08942065, 0.50819672, 0.99747793])}

## Making report table between two models

In [0]:
report = pd.DataFrame(index=list(rfc_scores.keys())[2:], columns=['Decision Tree', 'Random Forest'])
for key in report.index:
  report.loc[key] = [np.mean(scores[key]), np.mean(rfc_scores[key])]

report['Improvment'] = report['Random Forest'] - report['Decision Tree']
report *= 100
report = report.astype(float).round(1)

In [27]:
report

,Decision Tree,Random Forest,Improvment
test_accuracy,38.9,48.9,10.0
test_average_precision,64.4,66.2,1.8
test_recall,38.1,53.2,15.1
test_f1,32.7,49.9,17.1


## Saving the data for further use in visualization part

In [0]:
report.to_csv("ML_report.csv")

We save here better model of two

In [0]:
pickle.dump(rfc, open('RandomForestClasifier.model', 'wb'))